# Exploratory Analysis of a Monthly Dataset

In [1]:
# do the below to load the python utilities from py_depends

In [2]:
cd ..

/Users/svyatnakonechny/flight-delays


In [12]:
import sys
from py_depends.utils import *
from aux.dtypes import *

# sanity check - assert the import worked correctly
try:
    assert 'flight-delays' in os.getcwd()
except AssertionError:
    print('Doesn\'t look like the cd .. hack worked - restart the kernel and try again')

_First, some housekeeping. Reading in a sample file, 

In [4]:
%time jan_2018_df = unzipFile('./sample_data/On_Time_Marketing_Carrier_On_Time_Performance_Beginning_January_2018_2018_1.zip')

%time subs_jan_2018_df = subsetRenameRetype(jan_2018_df, new_cols)
%time subs_jan_2018_df = shapeDatetime(subs_jan_2018_df, ['crs_dep_time', 'dep_time','crs_arr_time', 'arr_time', 'first_dep_time', 'diverted_wheels_off_1', 'diverted_wheels_off_2', 'diverted_wheels_off_3'], 'flight_date')

/Users/svyatnakonechny/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (11,13,57,86,93) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [30]:
import pandas as pd

pd.to_datetime(jan_2018_df['CRSDepTime'],format= '%H%M' ).dt.time

ValueError: time data '1' does not match format '%H%M' (match)

In [31]:
jan_2018_df.CRSDepTime

0          615
1          618
2          830
3          650
4         1824
          ... 
621456    1731
621457    1940
621458     704
621459    1503
621460    1638
Name: CRSDepTime, Length: 621461, dtype: int64

In [36]:
pd.to_numeric(jan_2018_df['CRSDepTime'], errors='coerce').fillna(0)

0          615
1          618
2          830
3          650
4         1824
          ... 
621456    1731
621457    1940
621458     704
621459    1503
621460    1638
Name: CRSDepTime, Length: 621461, dtype: int64

In [35]:
jan_2018_df['CRSDepTime']

0          615
1          618
2          830
3          650
4         1824
          ... 
621456    1731
621457    1940
621458     704
621459    1503
621460    1638
Name: CRSDepTime, Length: 621461, dtype: int64